In [17]:
!pip install imutils


In [18]:
import os
import cv2
from tqdm import tqdm
from scipy.spatial import distance as dist
import imutils
import numpy as np
from sklearn.utils import shuffle
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# import mahotas

# Object detection:

did object detection using opencv by findig edges in the images the extract contours and from contours get image boxes

In [19]:
def findEdges(image):
    gray = cv2.GaussianBlur(image, (1, 1), 0)
    edged = cv2.Canny(gray, 100, 400)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    return edged


In [20]:
def getImgContours(edged):
    # find contours in the edge map
    contours = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=lambda x: cv2.contourArea(x))
    return contours



In [21]:
def getBoxes(contours, orig):
    boxes = []
    centers = []
    for contour in contours:
        box = cv2.minAreaRect(contour)
        box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        (tl, tr, br, bl) = box
        if (dist.euclidean(tl, bl)) > 0 and (dist.euclidean(tl, tr)) > 0:
            boxes.append(box)
    return boxes


# Loading Data:

Load_data function that loads images and labels from the Train and Test folders and combines the datasets to return Image-Label pairs 
combining the Train/Test folders and recreate the dataset .

In [22]:
class_names = ['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']
nb_classes = len(class_names)
image_size = (40,40)

In [23]:

def load_data():
    
    datasets = ['/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TRAIN',
                '/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TEST' ]    
    images = []
    labels = []

    # iterate through training and test sets
    count=0
    for dataset in datasets:

        # iterate through folders in each dataset
        
        for folder in os.listdir(dataset):

            if folder in ['EOSINOPHIL']: label = 1#[1, 0, 0, 0]
            elif folder in ['LYMPHOCYTE']: label = 2#[0, 1, 0, 0]
            elif folder in ['MONOCYTE']: label = 3#[0, 0, 1, 0]
            elif folder in ['NEUTROPHIL']: label = 4#[0, 0, 0, 1]

            # iterate through each image in folder
           
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):

                # get pathname of each image
                img_path = os.path.join(os.path.join(dataset, folder), file)

                # Open and resize the| img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                 # add padding to the image to better detect cell at the edge
                image = cv2.copyMakeBorder(image,10,10,10,10,cv2.BORDER_CONSTANT,value=[198, 203, 208])
                
                #thresholding the image to get the target cell
                image1 = cv2.inRange(image,(80, 80, 180),(180, 170, 245))
                
                # openning errosion then dilation
                kernel = np.ones((3, 3), np.uint8)
                kernel1 = np.ones((5, 5), np.uint8)
                img_erosion = cv2.erode(image1, kernel, iterations=2)
                image1 = cv2.dilate(img_erosion, kernel1, iterations=5)
                
                #detecting the blood cell
                edgedImage = findEdges(image1)
                edgedContours = getImgContours(edgedImage)
                edgedBoxes =  getBoxes(edgedContours, image.copy())
                if len(edgedBoxes)==0:
                    count +=1
                    continue
                # get the large box and get its cordinate
                last = edgedBoxes[-1]
                max_x = int(max(last[:,0]))
                min_x = int( min(last[:,0]))
                max_y = int(max(last[:,1]))
                min_y = int(min(last[:,1]))
                
                # draw the contour and fill it 
                mask = np.zeros_like(image)
                cv2.drawContours(mask, edgedContours, len(edgedContours)-1, (255,255,255), -1) 
                
                # any pixel but the pixels inside the contour is zero
                image[mask==0] = 0
                
                # extract th blood cell
                image = image[min_y:max_y, min_x:max_x]

                if (np.size(image)==0):
                    count +=1
                    continue
                # resize th image
                image = cv2.resize(image, image_size)                
                image = np.reshape(image,image.size )
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
    print(count)
    images = np.array(images, dtype = 'float32')
    labels = np.array(labels, dtype = 'uint8')

    return images, labels

In [24]:
images, labels = load_data()

100%|██████████| 623/623 [00:02<00:00, 246.34it/s]


298


# Normalization:


In [25]:
images = images/255.

# Adding intercept:

did object detection using opencv by findig edges in the images the extract contours and from contours get image boxes

In [26]:
images = np.append(np.ones((images.shape[0],1)),images,axis=1)

# Shuffling and Train-Test-Val split:

In [27]:

images, labels = shuffle(images, labels, random_state=10)
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size = 0.2)

In [28]:
from sklearn.svm import SVC
clf1 = SVC(kernel='poly',C=10,random_state=0,cache_size=2048*9)
clf1.fit(train_images,train_labels)


SVC(C=10, cache_size=18432, kernel='poly', random_state=0)

In [29]:
print("test_score",clf1.score(test_images,test_labels))
print("train_score",clf1.score(train_images,train_labels))

test_score 0.8506172839506173
train_score 0.9975298476739399
